# Exploratory Data Analysis - Financial Loans

The aim of this project is to gain insights into the portfolio of a loan company. In this notebook, we will be covering how the data is cleaned and prepared for analysis.

In [44]:
# Import modules and classes
from loan_data_analysis import *

## Getting Familiar With the Data
The raw data was obtained by connecting to a RDS Database and saving the data locally.

In [45]:
# Load the raw loan data into a DataFrame
loan_data = pd.read_csv('loans_data_raw.csv')

# Create a copy of the DataFrame for backup
loan_data_copy = loan_data.copy()

# Create an instance of DataFrameInfo for data exploration
df_info = DataFrameInfo(loan_data)

In [46]:
# Print the shape of the DataFrame
print("Shape of the DataFrame:")
print(df_info.print_shape())

Shape of the DataFrame:
(54231, 43)


We can see we are working with a medium sized dataset with over 50,000 records, and 43 variables being tracked.

#### Handling Data Types
First we will check the dataframe and then decide which columns will have to change data types.

In [47]:
# Describe the columns to understand their data types
print("Column Descriptions:")
print(df_info.describe_columns())

Column Descriptions:
id                               int64
member_id                        int64
loan_amount                      int64
funded_amount                  float64
funded_amount_inv              float64
term                            object
int_rate                       float64
instalment                     float64
grade                           object
sub_grade                       object
employment_length               object
home_ownership                  object
annual_inc                     float64
verification_status             object
issue_date                      object
loan_status                     object
payment_plan                    object
purpose                         object
dti                            float64
delinq_2yrs                      int64
earliest_credit_line            object
inq_last_6mths                   int64
mths_since_last_delinq         float64
mths_since_last_record         float64
open_accounts                    int64
tota

From this we can see the following columns are object data type which should be converted to __category__ data type:

- term
- home_ownership
- verification_status

and the following will be converted to an __ordered category__ data type.
- employment_length
- grade
- sub_grade

The date columns from object to __datetime__ data type, which includes the following:

- issue_date
- earliest_credit_line
- last_payment_date
- next_payment_date
- last_credit_pull_date

In [48]:
df_transform = DataTransform(loan_data)

# Convert columns to category type
category_column_names = ["term","grade","sub_grade","employment_length","home_ownership","verification_status"]
df = df_transform.convert_to_category(category_column_names)

# Convert columns to ordered category type
col_order = {
    "employment_length": ["< 1 year", "1 year", "2 years", "3 years", "4 years","5 years", "6 years","7 years", "8 years","9 years", "10+ years"],
    "grade": ["A","B","C","D","E","F"],
    "sub_grade": ["A1","A2","A3","A4","A5","B1","B2","B3","B4","B5","C1","C2","C3","C4","C5","D1","D2","D3","D4","D5","E1","E2","E3","E4","E5","F1","F2","F3","F4","F5"]
}
df = df_transform.convert_to_ordered_category(col_order)

# Convert columns to datetime type
date_column_names = ["issue_date","earliest_credit_line","last_payment_date","next_payment_date","last_credit_pull_date"]
df = df_transform.convert_to_date(date_column_names)

print(df.shape())

TypeError: 'tuple' object is not callable

#### Exploring the Categorical Data

In [ ]:
# Calculate distinct values and mode of catagorical data
cat_stats = df_info.count_distinct_values_and_mode()

 # Convert stats dictionary to a DataFrame for better readability
cat_stats_df = pd.DataFrame(cat_stats).T  # Transpose the table so columns are rows
cat_stats_df.index.name = 'Variable' 
print(cat_stats_df)

From this we know all the loans are personal loans, most are for mortgages and fully paid. The loan terms can either be 60 or 36 months, the most common being 36 months.

#### Exploring the Numeric Data

In [ ]:
# Extract and format statistics for numeric columns
stats_dict = df_info.extract_statistics() # This returns a dictionary with Pandas Series
formatted_stats_dict = {key: stats_dict[key].apply(lambda x: f"{x:,.2f}") for key in stats_dict}

 # Convert stats dictionary to a DataFrame for better readability
num_stats_df = pd.DataFrame(formatted_stats_dict) 
num_stats_df.index.name = 'Variable' 
print(num_stats_df)

Key Averages
- loan_amouunt £13,333.08
- total_payment £12,079.21 which is below the loan_amount, suggesting they are working at a loss.
- int_rate 13.51% this is high interest rate for a personal loan.

Some strange results can be seen by these preliminary calculations, the standard deviation of the loan amount is 8,082.20. With some cleaning of the data,  more nuanced insights that we can have confidence in can be gained.

## Handling Null Values
Handling null values is important as they can effect the accuracy of future analysis and cause errors in our code.

#### Null Value Summary

First let us calculate the count and percentage of null values in each column to give us an idea of what we are working with.


In [ ]:
#Count null values
null_summary = df_info.count_null_values()
print("Null Value Summary:")
print(null_summary)
print(f"\n Number of columns with null values {(null_summary['count'] != 0).sum()}")

There are 11 variables with null values ranging from 86% to less than %, each one will have to be handled on a case by case basis.

#### Dropping Columns

Variables with a null value percentage greater than 50% will be dropped as dropping the missing samples would reduce the dataset too much. This includes:
- mths_since_last_delinq
- mths_since_last_record  
- next_payment_date 
- mths_since_last_major_derog 

In [ ]:
# Create instance of DataFrameTransform using the transformed data
df_transform = DataFrameTransform(loan_data)

thresh_max = 50  # threshold percentage 
df_transform.drop_columns(null_summary,thresh_max)

#### Dropping Rows

The following variables have null values percentages of below 1%:
- last_payment_date (0.13%)
- last_credit_pull_date (0.01%)
- collections_12_mths_ex_med (0.09%)

Removing the null values of these variables will not reduce the size of the data set too much so it is easier to  drop the rows with null nvalues, additionally the variables with datetime data type (last_payment_date and last_credit_pull_date) would be difficult to impute and not provide much more insight.

The employment_length variable has a null percentage of 3.9% which is worth considering imputing. First lets look at the spread of the data by plotting a histogram to check the distribution.

In [ ]:
# Sort the DataFrame by the 'employment_length' column
df_sorted = df_transform.df.sort_values(by='employment_length')
x_val = df_sorted['employment_length']
# Plot the histogram
employment_length_inst = Plotter(df_sorted)
employment_length_hist = employment_length_inst.histogram(x_val,"Employment Length", "Employment Length Histogram")

The histogram shows that the mode is 10+ years of employment, which makes sense as that captures more years than one year. Since it does not have a normal distribution imputing it with the mode would introduce bias and the employment_length cannot be infered from other data in this dataset. The best option is to drop the rows with null values. 

Setting a threshold of 4% ensures the employment_length, last_payment_date, last_credit_pull_date, collections_12_mths_ex_med are dropped.

In [ ]:
thresh_min = 4 

df_transform.drop_rows(null_summary,thresh_min)

#### Inferring missing data

Funded Amount

The funded_amount is the same as the loan amount so we can imptute the data using the corresponding loan data.

In [ ]:
# Impute missing funded_amount values with corresponding loan_amount values
df_transform.df['funded_amount'] = df_transform.df['funded_amount'].fillna(value=df_transform.df['loan_amount'])
null_summary.loc['funded_amount',:]

Term

The term can only be a 36 month contract of 60 month contract, the missing term values can be calculated by multiplying the instalment by 36 or 60 and choosing the one closer to the loan amount.

In [ ]:
# Calculate the predited loan amounts
loan_36 = df_transform.df['instalment'] * 36
loan_60 = df_transform.df['instalment'] * 60
# Calculate the difference between the real loan and predicted loan
dif_36 =  df_transform.df['loan_amount'] - loan_36
dif_60 =  df_transform.df['loan_amount'] - loan_60
# Create a DataFrame to display the series and the comparison result
df_comparison = pd.DataFrame({
    '36': dif_36,
    '60': dif_60
})

# Determine if the term is 36 or 60 months
df_comparison['term'] = df_comparison.apply(lambda row: '60' if row['36'] > 0 else '36', axis=1)

df_check = pd.DataFrame({
    "calc_term": df_comparison['term'],
    'real_term' : df_transform.df['term'],

})
print(df_check.head(9))

df_transform.df['term'] = df_comparison['term']

print('number of term null values',df_transform.df['term'].isnull().sum())

#### Mean Imputation

The interest rate can be imputed using the mean/median as the null percentage is less than 10%, as the bias and increase of variance will be negligble.

In [ ]:

x_int_rate = df_transform.df['int_rate'] #x axis values
data_vis = Plotter(df_transform.df)
int_rate_hist = data_vis.histogram(x_int_rate,"Interest Rate", "Interest Rate Histogram")

In [ ]:
print(f'The median of int_rate is {df_transform.df["int_rate"].median()}')
print(f'The mean of int_rate is {df_transform.df["int_rate"].mean()}')

The interest rate has a right skewed distribution, which typically increases the mean as seen in our calculations. A median imputation is  the better option in this scenario.

It could be interesting to see a correlation matrix to determine which variable is mostly strongly correlated and doing a regression based off that and comparing the result to the median regression, testing wheather our assumption that a null percentage value of less than 10% is an acceptable threshold.

In [16]:
# Impute int_rate with median:
df_transform.df['int_rate'] = df_transform.impute_median('int_rate')

#### Visualising the Change in Null Values

Using the missingno package we are able to visualise the null values in the dataset. Every white line is a missing record.

Snapshot of dataset before cleaning.

In [ ]:
msno.matrix(loan_data_copy)

Snapshot of dataset after cleaning.

In [ ]:
msno.matrix(df_transform.df)

We can see there are no null recrods left in the dataset.

Save dataframe for future analysis.

In [ ]:
# Save data to CSV
df_transform.df.to_csv('loans_data_clean.csv', index=False)
print("Data saved to loans_data_clean.csv")


## Handling Skewewd Data
Skewed data can lead to biased models and inaccurate results, so it's important to address this issue before proceeding with any analysis.

#### Visualising the Skewed Data

In [ ]:

# Select numeric columns to be analysed for skewness
numeric_cols = ['loan_amount', 'funded_amount', 'funded_amount_inv',
       'int_rate', 'instalment', 'annual_inc', 'dti', 'delinq_2yrs', 
       'inq_last_6mths', 'open_accounts', 'total_accounts', 'out_prncp',
       'out_prncp_inv', 'total_payment', 'total_payment_inv',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee','last_payment_amount',
       'collections_12_mths_ex_med']

# Columns not to be analysed for skewness
non_skew_features = [col for col in df_transform.df.columns if col not in numeric_cols] 

#facetgrid plot
sns.set_theme(font_scale=0.7)
f = pd.melt(df_transform.df, value_vars=numeric_cols)
g = sns.FacetGrid(f, col="variable",  col_wrap=3, sharex=False, sharey=False)
g = g.map(sns.histplot, "value", kde=True)

Calculate skewness and determine a threshold for the skewness of the data, over which a column will be considered skewed.

In [ ]:
df = df_transform.df 
# Calculate skewness
skewness = df[numeric_cols].skew()

print(f"Skewness values:\n{skewness}")



All variables have a positive skew. A normal distribution has a skew of 0, suggesting that our data is not normally distributed. If you're dealing with normally distributed data, a skewness threshold of ±0.5 is often used. For less strict symmetry, a threshold of ±1 is reasonable.

In [ ]:
# Define threshold
threshold = 1  # Adjust as needed

# Identify highly skewed columns
highly_skewed_cols = skewness[abs(skewness) > threshold]

print("\nHighly skewed columns:\n", highly_skewed_cols)
col_names = highly_skewed_cols.index.tolist()

### Perform transformations 

on these columns to determine which transformation results in the biggest reduction in skew. 
Create the the method to transform the columns in your DateFrameTransform class.


In [ ]:
transformed_df = df_transform.skew_transform(col_names)

## Removing Outliers

Outliers are samples which lie outside the range of expected values. They can be spotted by visual inspection and calculated by using methods such as the Interquartile Range Method. However for variables that have many zero values such as the delinq_2years which measures how many delinquencies a customer has had in the past 2 years. Most customers have no delinquencies so calculating outliers for these variables must be adapted to account for the zero values.

#### Visualising Outliers

In [ ]:
#facetgrid plot
sns.set_theme(font_scale=0.7)
f = pd.melt(df_transform.df, value_vars=numeric_cols)
g = sns.FacetGrid(f, col="variable",  col_wrap=3, sharex=False, sharey=False)
g = g.map(sns.boxplot, "value")

The plots show the outliers as the circles outside of the whiskers of the boxplot. We can see for variables that have many zero values that there are many outliers.

#### Removing Outliers

The variables have been split into two groups. The first group will have outliers removed using the traditional IQR Method where the threshold is +/- 1.5 *IQR. The second group is for variables that have meaningful zero values, and the thresholds are relaxed  to +/- 3 * IQR. 

In [25]:
#sort columns into different methods to remove outliers
IQR_cols = ['loan_amount', 'funded_amount', 'funded_amount_inv',
       'int_rate', 'instalment', 'annual_inc', 'dti', 'open_accounts', 'total_accounts',
       'total_payment', 'total_payment_inv',
       'total_rec_prncp', 'total_rec_int', 
       'last_payment_amount',
       ]

for col in IQR_cols:
    df_no_outliers = df_transform.remove_outliers_iqr(col)
    
#'out_prncp', 'out_prncp_inv' , zero means nothing outstanding

IQR_mod = [ 'delinq_2yrs', 'inq_last_6mths',
           'out_prncp', 'out_prncp_inv',
           'total_rec_late_fee', 'recoveries',
           'collection_recovery_fee',
           'collections_12_mths_ex_med'
           ]

for col in IQR_mod:
    df_no_outliers = df_transform.remove_outliers_modified_iqr(col)

In [ ]:

df_original = pd.read_csv('loans_data_clean.csv') # Copy before removing outliers/skew transformations

# Apply outlier removal (Assuming remove_outliers_iqr is implemented in df_transform)

col = 'loan_amount'
df_no_outliers_copy = df_no_outliers.copy()

# Prepare Data for Boxplot (Melt for better visualization)
df_original["Outlier_Status"] = "Before Removal"
df_no_outliers_copy["Outlier_Status"] = "After Removal"

df_combined = pd.concat([df_original, df_no_outliers_copy])  # Combine both datasets

# Plot Boxplots
plt.figure(figsize=(8, 6))
sns.boxplot(x="Outlier_Status", y=col, data=df_combined)

# Add labels
plt.title(f"Comparison of { col } Before and After Outlier Removal")
plt.xlabel("Dataset")
plt.ylabel(col)
plt.show()

## Dropping Overly Correlated Columns

"Highly correlated columns in a dataset can lead to multicollinearity issues, which can affect the accuracy and interpretability of models built on the data."

#### Visualise the correlation matrix

In [ ]:
# Filter for numeric columns only
numeric_df = df_no_outliers.select_dtypes(include=['number'])

# Compute the correlation matrix
corr_matrix = numeric_df.corr()

# Visualize the correlation matrix using a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix')
plt.show()

We can see two clusters of very high correlation (>0.9). The first in the top right between loan_amount, funded_amount, funded_amount_inv, and instalment. This makes sense as the latter three are based on the first. The second cluster in the bottom left is between total_payment, total_payment_inv, total_rec_prncp which is expected. Another obvious insight from the heatmap is that out_prncp and put_prncp_inv are perfectly correlated.

### Identify Highly Correlated Columns

Based on the heatmap I will choose a threshold of 0.9

In [ ]:
# Set a correlation threshold
threshold = 0.9

# Find pairs of columns with correlation above the threshold
high_corr_pairs = np.where(np.abs(corr_matrix) > threshold)

# Filter out pairs where the correlation is not with itself
high_corr_pairs = [(corr_matrix.columns[x], corr_matrix.columns[y]) 
                   for x, y in zip(*high_corr_pairs) 
                   if x != y and x < y]

# Display the highly correlated pairs
print("Highly correlated pairs:")
for pair in high_corr_pairs:
    print(pair)

#### Step 3: Decide Which Columns to Remove

In [49]:
# Create a set of columns to remove
columns_to_remove = set()

# Iterate through the highly correlated pairs and remove the second columns
for col1, col2 in high_corr_pairs:
    columns_to_remove.add(col2)

# Display the columns to remove
print("Columns to remove:", columns_to_remove)

Columns to remove: {'total_rec_prncp', 'out_prncp_inv', 'funded_amount_inv', 'total_payment_inv', 'instalment', 'funded_amount', 'member_id'}


A threshold of 0.9 removes 7 columns:
funded_amount, funded_amount_inv, instalment, total_payment_inv, total_rec_prncp, out_prncp_inv, member_id.

In [ ]:
# Drop the columns from the DataFrame
df_reduced = df_no_outliers.drop(columns=columns_to_remove)

# Display the reduced DataFra
print(df_reduced)

Now the dataset is ready to be used for analysis.